In [1]:
# load the dataset
from tensorflow.keras.datasets import mnist

In [2]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train/255.0
X_test = X_test/255.0

11490434/11490434 [==============================] - 2s 0us/step


In [3]:
!pip install keras_tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 1.9 MB/s eta 0:00:00


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dropout, Dense
from tensorflow.keras.optimizers import Adam
import keras_tuner
from keras_tuner.tuners import RandomSearch
print('library imported')

library imported


In [5]:
def build_model(hp):
    model = Sequential()
    model.add(Flatten(input_shape=(28, 28)))
    for i in range(hp.Int('num_layers', 2, 20)): # search for layers between 2 to 20
        model.add(Dense(units=hp.Int('units_'+str(i), min_value=32, max_value=512, step=32), activation='relu'))
        # search for number of units
        model.add(Dropout(hp.Float('rate'+str(i), min_value=0.0, max_value=0.5, step=0.1)))
    model.add(Dense(10, activation='softmax'))
    model.compile(optimizer=Adam(hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [6]:
# tuner
tuner = RandomSearch(build_model, objective='val_accuracy',
                     max_trials=3, # tuner will explore 3 diff set of hyperparameters
                     max_retries_per_trial=2,
                     directory='my_dir', project_name='Hyper_tuner1')

In [7]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 3 Complete [00h 01m 55s]
val_accuracy: 0.11349999904632568

Best val_accuracy So Far: 0.9430000185966492
Total elapsed time: 00h 05m 09s


In [8]:
tuner.results_summary()

Results summary
Results in my_dir/Hyper_tuner1
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 0 summary
Hyperparameters:
num_layers: 18
units_0: 416
rate0: 0.4
units_1: 32
rate1: 0.2
learning_rate: 0.0001
units_2: 32
rate2: 0.0
units_3: 32
rate3: 0.0
units_4: 32
rate4: 0.0
units_5: 32
rate5: 0.0
units_6: 32
rate6: 0.0
units_7: 32
rate7: 0.0
units_8: 32
rate8: 0.0
units_9: 32
rate9: 0.0
units_10: 32
rate10: 0.0
units_11: 32
rate11: 0.0
units_12: 32
rate12: 0.0
units_13: 32
rate13: 0.0
units_14: 32
rate14: 0.0
units_15: 32
rate15: 0.0
units_16: 32
rate16: 0.0
units_17: 32
rate17: 0.0
Score: 0.9430000185966492

Trial 1 summary
Hyperparameters:
num_layers: 11
units_0: 192
rate0: 0.30000000000000004
units_1: 512
rate1: 0.0
learning_rate: 0.01
units_2: 224
rate2: 0.2
units_3: 448
rate3: 0.30000000000000004
units_4: 192
rate4: 0.0
units_5: 160
rate5: 0.4
units_6: 512
rate6: 0.30000000000000004
units_7: 64
rate7: 0.0
units_8: 32
rate8: 0.1
units_9: 160
rate9: 0.0

In [9]:
best_hps = tuner.get_best_hyperparameters()[0]
model = tuner.hypermodel.build(best_hps)
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 dense_18 (Dense)            (None, 416)               326560    
                                                                 
 dropout_17 (Dropout)        (None, 416)               0         
                                                                 
 dense_19 (Dense)            (None, 32)                13344     
                                                                 
 dropout_18 (Dropout)        (None, 32)                0         
                                                                 
 dense_20 (Dense)            (None, 32)                1056      
                                                                 
 dropout_19 (Dropout)        (None, 32)               

In [10]:
model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Epoch 1/5
1875/1875 [==============================] - 22s 9ms/step - loss: 1.4291 - accuracy: 0.4564 - val_loss: 0.5636 - val_accuracy: 0.8570
Epoch 2/5
1875/1875 [==============================] - 17s 9ms/step - loss: 0.5538 - accuracy: 0.8361 - val_loss: 0.3353 - val_accuracy: 0.9157
Epoch 3/5
1875/1875 [==============================] - 16s 9ms/step - loss: 0.3941 - accuracy: 0.8915 - val_loss: 0.2606 - val_accuracy: 0.9381
Epoch 4/5
1875/1875 [==============================] - 21s 11ms/step - loss: 0.3237 - accuracy: 0.9130 - val_loss: 0.2233 - val_accuracy: 0.9459
Epoch 5/5
1875/1875 [==============================] - 19s 10ms/step - loss: 0.2771 - accuracy: 0.9249 - val_loss: 0.2005 - val_accuracy: 0.9508
